<img src="https://www.cruxbv.nl/contents/uploads/nieuws/56_1.cems_header4.png"
alt="CEMS logo"
style="float: left; margin-right: 10px;" />

# VibraCore API

A benchmark calculation becomes obsolete when you have the power to do a risk assessment for all nearby buildings of your construction site within a fraction of the time. Now you can try it yourself. Interested? Read on and find out!

- **Website**: 	https://cemsbv.nl/products/vibracore
- **API**: 		https://crux-nuclei.com/api/vibracore/v1/ui/
- **NUCLEI**: 	https://cemsbv.github.io/nuclei/
- **sign-up**: 	https://crux-nuclei.com/sign-up

VibraCore is an API service that automates the risk management of building damage during vibration works, such as the installation of sheet piles or driven piles. According to CUR 166 the maximum vibration velocity is predicted. Based on the attributes of the nearby buildings, the failure vibration velocity is calculated (according to SBR A). If the check fails, the building has an unacceptable risk (according to SBR A) of being damaged by the installation of the piles or sheet piles.

Authentication is needed to call one of the endpoints. To get the credentials needed, please [sign up](https://crux-nuclei.com/sign-up) and validate your email address. To guide inexperienced programmers, we created a library that helps you to communicate with our API. This library is called [`cems-nuclei`](https://cemsbv.github.io/nuclei/) and is available on pypi.

To make sure that everything is set up correctly, you can use the following code block to call the API. After providing your credentials, you should get `Service alive`.

In [1]:
import nuclei

APP = "vibracore"
schema = {}

print(nuclei.call_endpoint(APP, "/healthcheck", schema))

Service alive


To perform a risk management according to SBR A and CUR 166 you require information of the building, the source and soil attributes. This information is bundled into the schema of the API call. The [API documentation](https://crux-nuclei.com/api/vibracore/v1/ui/) provides a detailed description of every attribute.

Below you will find a small example to generate a report. Please note that we create one dummy building at 12 meters from the installation location. To add more buildings, simply extended the array.

In [2]:
schema = {
    "building_info": [
        {
            "geometry": {
                "coordinates": [[[12, 0], [12, 0], [12, 0]]],
                "type": "Polygon",
            },
            "metadata": {"ID": "1234"},
            "properties_cur": {
                "building_part": "vloer",
                "foundation_element": "staal",
                "installation_type": "trillen",
                "material": "beton",
                "safety_factor": 0.05,
                "vibration_direction": "vertical",
            },
            "properties_sbr_a": {
                "category": "one",
                "frequency": 30,
                "monumental": True,
                "structural_condition": "gevoelig",
                "vibration_sensitive": True,
                "vibration_type": "continu",
            },
        }
    ],
    "prediction": {
        "attenuation_constant": 1.1,
        "force": 500,
        "measurement_type": "indicatief",
        "methode_safety_factor": "CUR",
        "references_velocity": 0.9,
        "variation_coefficient": 0.02,
    },
    "validation": {"source_location": {"coordinates": [0, 0], "type": "Point"}},
    "report_properties": {
        "author": "C. EMS",
        "project_name": "Teaser",
        "project_number": "987654",
        "project_remark": "This is a dummy calculation.",
    },
}

response = nuclei.call_endpoint(APP, "/cur166/report", schema)

print(response)

{'id': '6763c040-2963-40b2-bb88-9043556d1e8c'}


Most of the endpoints of VibraCore provide a task ID. This means that the job is running in the background. To fetch the job results, you need to call a different endpoint. This call provides the status of the job, if still pending.

In [3]:
pdf = nuclei.call_endpoint(APP, "/get-task", response)

if isinstance(pdf, bytes):
    display({"application/pdf": pdf}, raw=True)
else:
    print(pdf)

Task with id 6763c040-2963-40b2-bb88-9043556d1e8c is still running/pending.


Congratulations! 🚀 You generated a report that holds the risk assessment of possible building damage as a result of placing a sheet pile. Depending on your setup, either bytes or a PDF is displayed at this point.

By adding more buildings to the `building_info` attribute of the schema, more buildings are validated. To use the full potential of the API, combine VibraCore with open source data, for example the BAG (key registry address and buildings). In doing so, we are able to perform a validation on the entire neighborhood of the project location, providing a higher detail on risk management.